In [1]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import pandas as pd 
import six


In [2]:
# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: Hello, world!
Sentiment: 0.30000001192092896, 0.30000001192092896


In [3]:
def category_text(text):
    """Classifies content categories of the provided text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')
    
    #print(type(text.encode('utf-8')))
    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)
    
    try: 
        categories = client.classify_text(document).categories
    except:
        categories = []
    
    return categories
'''    
    #print(type(categories))
    if  type(categories) == "google.protobuf.pyext._message.RepeatedCompositeContainer" :
        for category in categories:
            print(category['name'])
'''    
    

'    \n    #print(type(categories))\n    if  type(categories) == "google.protobuf.pyext._message.RepeatedCompositeContainer" :\n        for category in categories:\n            print(category[\'name\'])\n'

In [4]:
def entities_text(text):
    """Detects entities in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects entities in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    entities = client.analyze_entities(document).entities

    # entity types from enums.Entity.Type
    entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
                   'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')
    '''
    for entity in entities:
        print('=' * 20)
        print(u'{:<16}: {}'.format('name', entity.name))
        print(u'{:<16}: {}'.format('type', entity_type[entity.type]))
        print(u'{:<16}: {}'.format('metadata', entity.metadata))
        print(u'{:<16}: {}'.format('salience', entity.salience))
        print(u'{:<16}: {}'.format('wikipedia_url',
              entity.metadata.get('wikipedia_url', '-')))
    '''
    return entities

In [18]:
courses = pd.read_csv("../data/data.csv", engine='python')
#courses.set_index('ID', inplace=True)
courses['category'] =''
courses['entity_type'] =''
courses['entity_name'] =''
courses.head()

,ID,Name,Description,Instructor,category,entity_type,entity_name
0,590DV,Data Visualization,Data visualization is crucial to conveying inf...,Matthew Turk,,,
1,590DW,Data Warehousing and BI,This course examines the construction of a dat...,Michael Wonderlich,,,
2,590ED1,Exhibit Design & Installation,This course meets the first 8 weeks of the sem...,Walter Wilson,,,
3,590ED2,Exhibit Design & Installation,This course meets the last 8 weeks of the seme...,Walter Wilson,,,
4,590ELO,E-Learning,2 or 4 credits: Provides an introduction to a ...,Melissa Wong,,,


In [19]:
def get_category_list(text_data):
    category=category_text(text_data)
    category_list =[]
    
    #print(category)
        
    for cat in category:
        #if cat.confidence > 0.50:
        category_list.append(cat.name)
    
    return category_list
    
    

In [20]:
def get_entity_list(text_data):
    entities=entities_text(text_data)
    entities_list =[]
    entity_type = []
    #print(category)
    
    # entity types from enums.Entity.Type
    entity_enum= ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
                   'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')    
    for entity in entities:
        #if cat.confidence > 0.50:
        entities_list.append(entity.name)
        #print(entity_enum[entity.type])
        entity_type.append(entity_enum[entity.type])
    return entities_list, entity_type

In [21]:
#get_entity_list(courses["Description"][0])

In [22]:
for i in courses.index:
    desc = courses.loc[i]['Description']
    #print(desc)
    cat_list = get_category_list(desc)
    ent_list,ent_type = get_entity_list(desc)
    #print(cat_list)
    courses.loc[i]['category'] = cat_list
    courses.loc[i]['entity_type'] = ent_type
    courses.loc[i]['entity_name'] = ent_list
    

In [23]:
#courses

In [ ]:
courses.to_csv('../data/syllabus_category.csv')

In [24]:
courses_cat = courses 
new_df = courses

In [25]:
gen_cat = ['Course Information','Class Schedule Information', 'fee', 'fees','Meets','students','Laptop','data','text'
           ,'libraries','Focuses','introduction' ,'Prerequisite','information',
           'Consideration', 'settings', 'responsibilities', 'Class materials fee','credit',
           'credits','Class Schedule.','field trip fee','course','others','INFO','topics courses',
           'INFO 491','CPSC 491','standing','project', 'use','uses','people','selection',
          'AU','needs', 'interests','outreach','overview','practice','content','portion',
          'curriculum','parts','look','part','sections','Students','projects','software', 'Variety', 'perspective', 'course materials fee',
          'understanding']

In [26]:
for ind in courses.index:
    ent_list = courses.loc[ind]["entity_name"]
    new_list = []
    for e in ent_list:
        if e not in gen_cat:
            new_list.append(e)
    new_df.loc[ind]['entity_name'] = new_list
    #print(new_list)

In [27]:
#course_cat = pd.read_csv('../data/desc_category.csv')
len(courses)

graph_df = pd.DataFrame(data=None, columns=['Course1', 'Course2','Common Entities'])



In [28]:
new_df.head()

,ID,Name,Description,Instructor,category,entity_type,entity_name
0,590DV,Data Visualization,Data visualization is crucial to conveying inf...,Matthew Turk,[/Computers & Electronics/Enterprise Technology],"[OTHER, OTHER, CONSUMER_GOOD, OTHER, EVENT, OT...","[Data visualization, models, observations, inv..."
1,590DW,Data Warehousing and BI,This course examines the construction of a dat...,Michael Wonderlich,"[/Jobs & Education/Education, /Business & Indu...","[OTHER, OTHER, ORGANIZATION, OTHER, OTHER, OTH...","[business intelligence system, data warehouse,..."
2,590ED1,Exhibit Design & Installation,This course meets the first 8 weeks of the sem...,Walter Wilson,"[/Jobs & Education/Education, /Arts & Entertai...","[OTHER, OTHER, OTHER, EVENT, OTHER, OTHER, OTH...","[idea, culmination, exhibition, planning, show..."
3,590ED2,Exhibit Design & Installation,This course meets the last 8 weeks of the seme...,Walter Wilson,"[/Jobs & Education/Education, /Arts & Entertai...","[OTHER, OTHER, OTHER, EVENT, OTHER, OTHER, OTH...","[idea, culmination, exhibition, planning, show..."
4,590ELO,E-Learning,2 or 4 credits: Provides an introduction to a ...,Melissa Wong,[/Jobs & Education/Education],"[OTHER, OTHER, OTHER, WORK_OF_ART, EVENT, OTHE...","[variety, learning methods, videos, courses, m..."


In [29]:
for i in courses.index:
    cid1= new_df.loc[i]['ID']
    a = set(new_df.loc[i]["entity_name"])
    a_minus_gen_cat = a.difference(set(gen_cat))
    
    
    for j in range(i+1,len(courses) ):
        cid2 = new_df.loc[j]['ID']
        
        b = set(new_df.loc[j]["entity_name"])
        common = a_minus_gen_cat.intersection(b)
        if len(list(common)) != 0 :
            temp= pd.DataFrame([[cid1,cid2,list(common)]], columns=graph_df.columns)
            graph_df = graph_df.append(temp, ignore_index=True,verify_integrity=False)
        #print(temp.columns)
    #print(graph_df)
    #break
        
        
        

In [31]:
graph_df
graph_df.to_csv('pure_desc.csv')

In [17]:
graph_df.to_csv('Graph_syllabus.csv')

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 

%matplotlib inline

In [ ]:
rand_idx = np.random.randint(0,len(graph_df),size=10)
rand_idx

In [ ]:
#A = graph_df['Course2'].isin(['490DB','490DT','590DV'])
A=graph_df.loc[rand_idx]

In [ ]:
#G = nx.from_pandas_dataframe(graph_df[A] ,'Course1','Course2')
G = nx.from_pandas_dataframe(graph_df ,'Course1','Course2')

In [ ]:
def dict_to_values(G, dict_data):
    return [dict_data[n] for n in G.nodes]

In [ ]:
graph_df.head()

In [ ]:
def plot_network(G, node_size_dict, factor=10, **kwargs):
    nx.draw(
        G, 
        pos=nx.spring_layout(G, k=0.1, scale=10),
        with_labels=True,
        node_size=[v*factor for v in dict_to_values(G, node_size_dict)],
        **kwargs
    )

In [ ]:
plot_network(G,nx.degree(G),factor=10,font_size=10, node_color="r", width=.25)


In [ ]:
### Centrality Measure
from utilities import get_all_node_metrics

cent_measure= {'degree': nx.degree(G),
               'betweenness': nx.betweenness_centrality(G),
               'closeness'  : nx.closeness_centrality(G)
              }

df_cm = get_all_node_metrics(G)

In [ ]:
df_cm.head()

In [ ]:
e = G.node
s_G = A["Course1"].values.tolist()
#sub_graph = graph_df[graph_df["Course1"].isin(['590DV','590DW','590PR','502AO'])]
#s_G = sub_graph["Course1"].values.tolist()

#len(s_G)

In [ ]:
H = G.subgraph(s_G)

In [ ]:
plot_network(H,nx.degree(H),factor=100,font_size=10, node_color=["b","g"], width=.25)

In [ ]:
#df_cm.sort_values(by=['degree'], axis=0)

In [ ]:
### Centrality Measure
from utilities import get_all_node_metrics

cent_measure= {'degree': nx.degree(G),
               'betweenness': nx.betweenness_centrality(G),
               'closeness'  : nx.closeness_centrality(G)
              }

df_cm = get_all_node_metrics(H)
df_cm.head()

In [33]:
df_cm.sort_values(by=["closeness"], ascending=False)

NameError: name 'df_cm' is not defined

In [34]:
graph_df.head()

,Course1,Course2,Common Entities
0,590DV,590DW,"[construction, aspects, IS 590, information sc..."
1,590DV,590ED1,"[aspects, IS 590, information sciences]"
2,590DV,590ED2,"[aspects, IS 590, information sciences]"
3,590DV,590ELO,"[models, aspects, IS 590, information sciences]"
4,590DV,590ET,"[aspects, IS 590, information sciences]"


In [32]:
import gensim
from nltk.tokenize import word_tokenize

In [36]:
### The above earlier two steps are done in one go in this code
gen_word_token_docs = [[ w.lower() for w in word_tokenize(str(text))] for text in graph_df['Common Entities']]

In [38]:
word_dictionary = gensim.corpora.Dictionary(gen_word_token_docs)

In [83]:
word_dictionary[10]

"'models"

In [41]:
corpus = [word_dictionary.doc2bow(gen_doc) for gen_doc in gen_word_token_docs]

In [88]:
word_dictionary.dfs[10]

55

In [80]:
new_df['frequency'] = word_dictionary.dfs.values()

In [89]:
new_df.to_csv("entity_frequency.csv")